In [1]:
import re
import os
from openpyxl import load_workbook

rootdir = "./compound_data"

collected_data = {
    
    "compound_name_dict" : [],
    "scf_parent_dict" : [],
    "zero_point_dict" : [],
    "free_energy_dict" : [],
    "bjd_dict" : [],
    "benzene_dict" : [],
    "bs2_dict" : []
    
}

def bjd(fin, file_data):
    try:
        if "bjd" in fin.name:
            bjdresult = re.search("SCF Done:\s+\S+\s=\s*(-?[0-9.]+)", file_data)
            collected_data["bjd_dict"].append(bjdresult.group(1))
    except:
        pass

def solvent(fin, file_data):
    try:
        if "benzene" in fin.name:
            benzene_result = re.search("SCF Done:\s+\S+\s=\s*(-?[0-9.]+)", file_data)
            collected_data["benzene_dict"].append(benzene_result.group(1))
    except:
        pass
            
def bs2(fin, file_data):
    try:
        if "bs2" in fin.name:
            bs2_result = re.search("SCF Done:\s+\S+\s=\s*(-?[0-9.]+)", file_data)
            collected_data["bs2_dict"].append(bs2_result.group(1))
    except:
        pass
        
        
def parent(fin, file_data):      
    try:
        if "bjd" not in fin.name and "benzene" not in fin.name and "bs2" not in fin.name:
            parent_result = list(re.findall("SCF Done:\s+\S+\s=\s*(-?[0-9.]*)", file_data))
            scf_parent = parent_result[-1]
            collected_data["scf_parent_dict"].append(scf_parent)
            zero_point_energies = re.search("zero-point Energies=\s+(-?[0-9.]+)", file_data)
            collected_data["zero_point_dict"].append(zero_point_energies.group(1))
            thermal_free_energies = re.search("thermal Free Energies=\s+(-?[0-9.]+)", file_data)
            collected_data["free_energy_dict"].append(thermal_free_energies.group(1))
    except:
        pass
        

for roots, directories, files in os.walk(rootdir):          
    
    for directory in directories:
        collected_data["compound_name_dict"].append(directory)
    
    for file in files:
        with open(os.path.join(roots, file), "r") as fin:    

            file_data = fin.read()
            
            bjd(fin, file_data)
            solvent(fin, file_data)
            bs2(fin, file_data)
            parent(fin, file_data)
                    

cells = {
    "compound_name_dict": ["A2", "A3", "A4", "A5"],
    "scf_parent_dict": ["B2", "B3", "B4", "B5"],
    "zero_point_dict": ["C2", "C3", "C4", "C5"],
    "free_energy_dict": ["D2", "D3", "D4", "D5"],      
    "benzene_dict": ["E2", "E3", "E4", "E5"],  # These two entries are swapped
    "bjd_dict": ["F2", "F3", "F4", "F5"],      # round compared to above
    "bs2_dict": ["G2", "G3", "G4", "G5"]
}

spreadsheetfile = "data.xlsx"
        
wb = load_workbook(spreadsheetfile)      
ws = wb.worksheets[0]        

for dict_name in collected_data:
    for i in range(0, 4):
        ws[cells[dict_name][i]] = (collected_data[dict_name][i])

wb.save(spreadsheetfile)
